In [1]:
import wave

from pyspeexdsp import EchoCanceller

# Constants
frame_size = 512

# Open the near-end and far-end audio files in binary mode
# near = wave.open("AEC_Mono_wDoubleTalk.wav", "rb")
near = wave.open("AEC_Mono.wav", "rb")
far = wave.open("FarEndMono.WAV", "rb")

# Ensure files are mono channel
if near.getnchannels() > 1 or far.getnchannels() > 1:
    print("Only support mono channel")
    exit()

# Prepare the output file
out = wave.open("output.wav", "wb")
out.setnchannels(near.getnchannels())
out.setsampwidth(near.getsampwidth())
out.setframerate(near.getframerate())

# Print file information
print(
    f"near - rate: {near.getframerate()}, channels: {near.getnchannels()}, length: {near.getnframes() / near.getframerate()}"
)
print(f"far - rate: {far.getframerate()}, channels: {far.getnchannels()}")

# Create an instance of the echo canceller
echo_canceller = EchoCanceller(frame_size, 4096, near.getframerate())
echo_canceller.set_echo_suppress(40)  # -40dB

# Define byte length for reading frames
in_data_len = frame_size
in_data_bytes = frame_size * 2  # Two bytes per sample (16-bit audio)
out_data_len = frame_size
out_data_bytes = frame_size * 2

# Process audio frames
while True:
    # Read frames from the near-end and far-end audio files
    in_data = near.readframes(in_data_len)
    out_data = far.readframes(out_data_len)

    # Stop if we've reached the end of either file
    if len(in_data) != in_data_bytes or len(out_data) != out_data_bytes:
        break

    # Process echo cancellation
    processed_data = echo_canceller.process(in_data, out_data)

    # Write processed frames to the output file
    out.writeframes(processed_data)

# Close files
near.close()
far.close()
out.close()


near - rate: 16000, channels: 1, length: 52.6186875
far - rate: 16000, channels: 1
